<a href="https://colab.research.google.com/github/iskra3138/colab_repo/blob/master/XAI_Visualization_Methods_TF1_x_Share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
!gsutil cp gs://iskra3138_mvtec/my_mvtec_tpumodel.h5 ./

In [0]:
%tensorflow_version 2.x

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from PIL import Image as pil_image
import cv2

import glob


import tensorflow as tf
from tensorflow.python.keras.models import model_from_json
from tensorflow.python.keras import backend as K
from tensorflow.python.framework import ops


import scipy
from scipy import ndimage
from skimage.measure import label, regionprops

%matplotlib inline

## CAM vs. Grad-CAM


In [0]:
model = tf.keras.models.load_model('my_mvtec_tpumodel.h5')

In [0]:
import os

AUTO = tf.data.experimental.AUTOTUNE

IMAGE_SIZE =  [224, 224]

validation_fns = 'gs://iskra3138_mvtec_tfrecords/valid.tfrecords'

def parse_tfrecord(example):
    features = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    label = example['label']
    #label = tf.one_hot(indices=label, depth=2)
    image = tf.io.decode_jpeg(example['image_raw'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32) / 255.0
    image = tf.image.resize(image, IMAGE_SIZE)
    
    return image, label

def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

#validation_dataset = load_dataset(validation_fns).shuffle(1000).batch(1000).prefetch(AUTO)

In [0]:
def NG_filter_fn(img, label):
  return tf.math.equal(label, 0)
def OK_filter_fn(img, label):
  return tf.math.equal(label, 1)

In [0]:
def make_class_sapling (fns, class_idx, batch_size) :
  if class_idx == 0 :
    NG_dataset = load_dataset(fns).filter(NG_filter_fn).shuffle(1000).batch(batch_size).prefetch(AUTO).repeat()
    NG_iter = NG_dataset.make_one_shot_iterator()
    return NG_iter
  else :
    OK_dataset = load_dataset(fns).filter(OK_filter_fn).shuffle(1000).batch(batch_size).prefetch(AUTO).repeat()
    OK_iter = OK_dataset.make_one_shot_iterator()
    return OK_iter
    

In [0]:
iter = make_class_sapling(validation_fns, 1, 9)
i, l = iter.get_next()
with tf.Session() as sess:
  print (sess.run(l))

In [0]:
import glob, random

img_paths=[]
for i in range(5):
  img_paths.append(random.choice(glob.glob('/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/*.jpg')))

In [0]:
img_paths

In [0]:
img_paths = ['/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_011_3_13.jpg',
 '/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_007_14_5.jpg',
 '/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_015_1_2.jpg',
 '/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_014_6_5.jpg',
 '/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_007_13_0.jpg']

In [0]:
path = '/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/cut_010_3_5.jpg'

In [0]:
'''def preprocess_input(img_path):
    img = pil_image.open(img_path).resize((224, 224))
    img_arr = np.asarray(img)[:, :, :3] / 255.
    img_tensor = np.expand_dims(img_arr, 0)
    
    return img_arr, img_tensor'''
  
def preprocess_input(img_path):  
  img_string = tf.io.read_file(img_path)
  img = tf.image.decode_jpeg(img_string)
  img = tf.image.convert_image_dtype(img, tf.float32)
  img_arr = tf.image.resize(img, (224, 224))
  with tf.compat.v1.Session() as sess:
    img_arr = sess.run(img_arr)
  img_arr /= 255.0
  img_tensor = np.expand_dims(img_arr, axis=0)  
  return img_arr, img_tensor
  
  

In [0]:
def generate_cam(model, img_path, class_idx):
    
    ## img_path -> preprocessed image tensor
    img_arr, img_tensor = preprocess_input(img_path)
    
    ## preprocessed image tensor -> last_conv_output, predictions
    get_output = K.function([model.layers[0].input], [model.get_layer('conv5_block3_out').output, model.layers[-1].output])
    [conv_outputs, predictions] = get_output([img_tensor])
    
    conv_outputs = conv_outputs[0, :, :, :]
    class_weights = model.layers[-1].get_weights()[0]
    
    ## generate cam
    cam = np.zeros(dtype=np.float32, shape=conv_outputs.shape[0:2])
    for i, w in enumerate(class_weights[:, class_idx]):
        cam += w * conv_outputs[:, :, i]
        
    cam /= np.max(cam)
    cam = cv2.resize(cam, (224, 224))
    
    return img_arr, cam, predictions    

In [0]:
# model.summary()

In [0]:
def generate_grad_cam(model, img_path, class_idx):

    ## img_path -> preprocessed image tensor
    img_arr, img_tensor = preprocess_input(img_path)

    ## get the derivative of y^c w.r.t A^k
    y_c = model.layers[-1].output.op.inputs[0][0, class_idx]
#     y_c = model.output[0, class_idx]
    
    layer_output = model.get_layer('conv5_block3_out').output
    #layer_output = model.layers[-4].output
    
    grads = K.gradients(y_c, layer_output)[0]
    gradient_fn = K.function([model.input], [layer_output, grads, model.layers[-1].output])
    
    conv_output, grad_val, predictions = gradient_fn([img_tensor])
    conv_output, grad_val = conv_output[0], grad_val[0]
    
    weights = np.mean(grad_val, axis=(0, 1))
    cam = np.dot(conv_output, weights)
    
    cam = cv2.resize(cam, (224, 224))
    
    ## Relu
    cam = np.maximum(cam, 0)
    
    cam = cam / cam.max()
    
    return img_arr, cam, predictions

In [0]:
def generate_bbox(img, cam, threshold):
    labeled, nr_objects = ndimage.label(cam > threshold)
    props = regionprops(labeled)
    return props

In [0]:
def build_guided_model():
    
    if 'GuidedBackProp' not in ops._gradient_registry._registry:
        @ops.RegisterGradient('GuidedBackProp')
        def _GruidedBackProp(op, grad):
            dtype = op.inputs[0].dtype
            return grad * tf.cast(grad > 0., dtype) * tf.cast(op.inputs[0] > 0., dtype)
        
    g = tf.get_default_graph()
    with g.gradient_override_map({'Relu': 'GuidedBackProp'}):
        
        
        new_model = tf.keras.models.load_model('my_mvtec_tpumodel.h5')
        
        return new_model

In [0]:
guided_model = build_guided_model()

In [0]:
img1, img2 = preprocess_input(img_paths[3])

In [0]:
model.predict(img2)

In [0]:
guided_model.predict(img2)

In [0]:
def guided_backprop(model, img, activation_layer):
    input_img = model.input
    #layer_output = model.layers[activation_layer_idx].output
    layer_output = model.get_layer('conv5_block3_out').output
    grads = K.gradients(layer_output, input_img)[0]
    gradient_fn = K.function([input_img], [grads])
    grads_val = gradient_fn([img])[0]
    return grads_val

def deprocess_image(x):
    x = x.copy()
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    
    # clip
    x += 0.5
    x = np.clip(x, 0, 1)
    
    ## RGB array
    x *= 255.
    
    x = np.clip(x, 0, 255).astype('uint8')
    
    return x

In [0]:
def generate_guided_grad_cam(guided_model, img_path, grad_cam):
    gb = guided_backprop(guided_model, img=preprocess_input(img_path)[1], activation_layer='conv5_block3_out')
    guided_grad_cam = gb * grad_cam[..., np.newaxis]
    
    return deprocess_image(guided_grad_cam)

In [0]:
##################

## img_path -> preprocessed image tensor
img_arr, img_tensor = preprocess_input(path)

## get the derivative of y^c w.r.t A^k
y_c = model.layers[-1].output.op.inputs[0][0, 0]
#     y_c = model.output[0, class_idx]

layer_output = model.get_layer('conv5_block3_out').output
#layer_output = model.layers[-4].output

grads = K.gradients(y_c, layer_output)[0]
#grads = tf.gradients(y_c, layer_output)[0]
gradient_fn = K.function([model.input], [layer_output, grads, model.layers[-1].output])

conv_output, grad_val, predictions = gradient_fn([img_tensor])
conv_output, grad_val = conv_output[0], grad_val[0]

weights = np.mean(grad_val, axis=(0, 1))
cam = np.dot(conv_output, weights)

#cam = cv2.resize(cam, (224, 224))

## Relu
cam = np.maximum(cam, 0)

#cam = cam / cam.max()


In [0]:
grad_val[:,:,0]

In [0]:
cam

In [0]:
cam

In [0]:
######
img, grad_cam, predictions = generate_grad_cam(model, path, 0)

In [0]:
##################
grad_cam

In [0]:
###################
predictions

In [0]:
################
img

In [0]:
fig, axes = plt.subplots(5, 6, figsize=(20, 15))

for i, s in enumerate(img_paths):
    img, cam, grad_cam, guided_grad_cam = None, None, None, None
    
    img_path = s
    class_idx = 0
    
    _, cam, _ = generate_cam(model, img_path, class_idx)
    img, grad_cam, predictions = generate_grad_cam(model, img_path, class_idx)
    guided_grad_cam = generate_guided_grad_cam(guided_model, img_path, grad_cam)
    img =img*255
    
    axes[i, 0].imshow(img)
    axes[i, 1].imshow(cam, cmap='jet')
    axes[i, 2].imshow(grad_cam, cmap='jet')
    axes[i, 3].imshow(img)
    axes[i, 3].imshow(cam, cmap='jet', alpha=0.5)
    axes[i, 4].imshow(img)
    axes[i, 4].imshow(grad_cam, cmap='jet', alpha=0.5)
    axes[i, 5].imshow(guided_grad_cam)
    
    
    axes[i, 0].axis('off')
    axes[i, 1].axis('off')
    axes[i, 2].axis('off')
    axes[i, 3].axis('off')
    axes[i, 4].axis('off')
    axes[i, 5].axis('off')
    
    axes[0, 0].set_title("image", fontsize=18)
    axes[0, 1].set_title("CAM", fontsize=18)
    axes[0, 2].set_title("Grad-CAM", fontsize=18)
    axes[0, 3].set_title("CAM", fontsize=18)
    axes[0, 4].set_title("Grad-CAM", fontsize=18)
    axes[0, 5].set_title("Guided-Grad-CAM", fontsize=18)
    
plt.tight_layout()
plt.show()

# CAM Visualization

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
model_file = 'my_mvtec_tpumodel.h5'

In [0]:
model = tf.keras.models.load_model(model_file)

In [0]:
model.summary()

### import matplotlib, numpy, cv2

In [0]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import keras
from keras.preprocessing import image
import keras.backend as K

from tensorflow.python.framework import ops
from PIL import  Image

import cv2

### import modules

In [0]:
#@title import Util code [Run Me!!!]

def load_image(path, target_size):
    img_string = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_string)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, target_size)
    img /= 255.0
    img = tf.expand_dims(img, axis=0)  
    return img
  
def deprocess_image(x):
    '''
    Same normalization as in:
    https://github.com/fchollet/keras/blob/master/examples/conv_filter_visualization.py
    '''
    x = x.copy()
    if np.ndim(x) > 3:
        x = np.squeeze(x)
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if tf.keras.backend.image_data_format() == 'channels_first':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# CAM Visualization

In [0]:
#@title import Grad-CAM Code [Run Me!!]
class GradCAM:
  def __init__(self, model, activation_layer):
    self.model = model
    self.activation_layer = activation_layer
    self.tensor_function = self._get_gradcam_tensor_function()

  # get partial tensor graph of CNN model
  def _get_gradcam_tensor_function(self):
    model_input = self.model.input
    #y_c = self.model.outputs[0].op.inputs[0][0, self.class_idx]
    #y_c = self.model.outputs[0].op.inputs[0]
    y_c = self.model.output
    A_k = self.model.get_layer(self.activation_layer).output
    
    tensor_function = tf.keras.models.Model([model_input], [A_k, y_c])
    return tensor_function

  # generate Grad-CAM
  def generate(self, input_tensor, class_idx):
    with tf.GradientTape() as tape:
      conv_outputs, predictions = self.tensor_function(input_tensor)
      loss = predictions[:, class_idx]

    output = conv_outputs[0]
    
    grads = tape.gradient(loss, conv_outputs)[0]    
    weights = np.mean(grads, axis=(0, 1))
    
    grad_cam = np.dot(output, weights)
    #grad_cam = np.zeros(output.shape[0: 2], dtype = np.float32)
    #for i, w in enumerate(weights):
    #    grad_cam += w * output[:, :, i]

    grad_cam = np.maximum(grad_cam, 0)
    grad_cam = cv2.resize(grad_cam, (224, 224))
    return grad_cam

In [0]:
#@title import Guided Grad-CAM Code [Run Me!!]
class Guided_GradCAM:
  def __init__(self, model, model_file, activation_layer, method='GuidedBackProp'):
    self.model = model
    self.model_file = model_file
    self.activation_layer = activation_layer

    if method == 'BackProp':
      self._register_backprop_gradient()
      self.guided_model = self._modify_graph('BackProp')
    elif method == 'DeconvNet':
      self._register_deconvnet_gradient()
      self.guided_model = self._modify_graph('DeconvNet')
    elif method == 'GuidedBackProp':
      self._register_guidedbackprop_gradient()
      self.guided_model = self._modify_graph('GuidedBackProp')
    else:
      sys.exit('method must be (BackProp, DeconvNet, GuidedBackProp)')

    self.tensor_function = self.get_tensor_function()

  # register gradient
  def _register_backprop_gradient(self):
    if "BackProp" not in ops._gradient_registry._registry:
      @ops.RegisterGradient("BackProp")
      def _BackProp(op, grad):
        dtype = op.inputs[0].dtype
        return grad * tf.cast(op.inputs[0] > 0., dtype)

  def _register_deconvnet_gradient(self):
    if "DeconvNet" not in ops._gradient_registry._registry:
      @ops.RegisterGradient("DeconvNet")
      def _DeconvNet(op, grad):
        dtype = op.inputs[0].dtype
        return grad * tf.cast(grad > 0., dtype)

  def _register_guidedbackprop_gradient(self):
    if "GuidedBackProp" not in ops._gradient_registry._registry:
      @ops.RegisterGradient("GuidedBackProp")
      def _GuidedBackProp(op, grad):
        dtype = op.inputs[0].dtype
        return grad * tf.cast(grad > 0., dtype) * tf.cast(op.inputs[0] > 0., dtype)
      
  # modify model graph
  def _modify_graph(self, name):
    g = tf.compat.v1.get_default_graph()
    
    with g.gradient_override_map({'Relu': name}):

      # get layers that have an activation
      layer_dict = [layer for layer in self.model.layers[1:]
                    if hasattr(layer, 'activation')]

      # replace relu activation
      for layer in layer_dict:
          if layer.activation == tf.keras.activations.relu:
              layer.activation = tf.nn.relu

      # re-instanciate a new model
      tf.keras.backend.reset_uids()
      new_model = tf.keras.models.load_model(self.model_file)
    return new_model
    '''        
    with g.gradient_override_map({'Relu': name}):       
      new_model = tf.keras.models.load_model(self.model_file)
      
    return new_model
    '''  
  # get partial tensor graph of CNN model
  def get_tensor_function(self):
    method = 'max'
    channel = 0
    input_img = self.guided_model.input
    layer_output = self.guided_model.get_layer(self.activation_layer).output
    
    if method == 'max':
        output = tf.keras.backend.max(layer_output, axis=3)
    elif method == 'one':
        output = layer_output[:, :, :, channel]
    else:
        sys.exit('method must be (max, one)')
       
    tensor_function = tf.keras.models.Model([input_img], [layer_output])
    
    return tensor_function
  
  # generate saliency map(gradient)
  def generate(self, input_tensor):
    x = tf.convert_to_tensor(input_tensor, dtype=tf.float32)
    with tf.GradientTape() as tape:
      tape.watch(x)
      layer_output = self.tensor_function(x)

    grads_val = tape.gradient(layer_output, x)[0]  
  
    return grads_val


In [0]:
layer_dict = [layer for layer in model.layers[1:]
                    if hasattr(layer, 'activation')]



In [0]:
layer_dict

In [0]:
# replace relu activation
for layer in layer_dict:
    if layer.activation == tf.keras.activations.relu :
      print (layer.activation)
      layer.activation = tf.nn.relu

In [0]:
model.get_layer(activation_layer).output

In [0]:
layer_dict = [layer for layer in model.layers[1:]
                     if hasattr(layer, 'activation')]
        
for layer in layer_dict:
    if layer.activation == tf.nn.relu:
        print(layer.activation)

### 클래스 생성

In [0]:
#@ title 변경시 자동실행 { run: "auto" }
method = "GuidedBackProp" #@param ["BackProp", "DeconvNet", "GuidedBackProp"]
activation_layer = "conv5_block3_out" #@param {type:"string"}


gradcam_generator = GradCAM(model, activation_layer)
guided_gradcam_generator = Guided_GradCAM(model, model_file, activation_layer, method=method)

## read NGimage path

In [0]:
import glob, random

img_paths=[]
for i in range(5):
  img_paths.append(random.choice(glob.glob('/gdrive/My Drive/MVTEC_EXP/DATA/Test/NG/*.jpg')))

In [0]:
img_paths

### generate Grad-CAM, Guided Grad-CAM

In [0]:
%%time
img_width = 224
img_height = 224

gradcams = []
guided_gradcams= []

for img_path in img_paths:
    img = load_image(path=img_path, target_size=(img_width, img_height))

    preds = model.predict(img)
    predicted_class = preds.argmax(axis=1)[0]
    print (predicted_class)

    gradcam = gradcam_generator.generate(img, predicted_class)
    gradcams.append(gradcam)
    
    guided_gradcam = guided_gradcam_generator.generate(img)
    guided_gradcams.append(guided_gradcam.numpy())

### Plot CAM

In [0]:
plt.figure(figsize=(20, 10))
left  = 0.125  # the left side of the subplots of the figure
right = 0.7    # the right side of the subplots of the figure
bottom = 0.2   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.02  # the amount of width reserved for blank space between subplots
hspace = 0.0   # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, right=right, bottom=bottom, top=top, wspace=wspace, hspace=hspace)

for idx in range(len(gradcams)):
    
    img = cv2.imread(img_paths[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    
    plt.subplot(3, 5, idx+1)
    plt.imshow(img)
    #plt.imshow(cams[idx], cmap="jet", alpha=.5)
    plt.axis('off')
    
    plt.subplot(3, 5, 5 + idx+1)
    plt.imshow(img)
    plt.imshow(gradcams[idx], cmap="jet", alpha=.5)
    plt.axis('off')
    
    #guided_img = guided_gradcams[idx] * gradcams[idx][..., np.newaxis]
    guided_img = guided_gradcams[idx]
    guided_img = deprocess_image(guided_img)
    #guided_img = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, guided_img, 1, 0) 
    guided_img = cv2.cvtColor(guided_img, cv2.COLOR_RGB2BGR)
    
   
    plt.subplot(3, 5, 10 + idx+1)
    plt.imshow(guided_img)
    plt.axis('off')

## read OK image path

In [0]:
import glob, random

img_paths=[]
for i in range(5):
  img_paths.append(random.choice(glob.glob('/gdrive/My Drive/MVTEC_EXP/DATA/Test/OK/*.jpg')))

In [0]:
img_paths

### generate cam(class activation map)

In [0]:
%%time
img_width = 224
img_height = 224

gradcams = []
guided_gradcams= []

for img_path in img_paths:
    img = load_image(path=img_path, target_size=(img_width, img_height))

    preds = model.predict(img)
    predicted_class = preds.argmax(axis=1)[0]
    print (predicted_class)

    gradcam = gradcam_generator.generate(img, predicted_class)
    gradcams.append(gradcam)
    
    guided_gradcam = guided_gradcam_generator.generate(img, gradcam)
    guided_gradcams.append(guided_gradcam.numpy())

### Plot CAM

In [0]:
plt.figure(figsize=(20, 10))
left  = 0.125  # the left side of the subplots of the figure
right = 0.7    # the right side of the subplots of the figure
bottom = 0.2   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.02  # the amount of width reserved for blank space between subplots
hspace = 0.0   # the amount of height reserved for white space between subplots
plt.subplots_adjust(left=left, right=right, bottom=bottom, top=top, wspace=wspace, hspace=hspace)

for idx in range(len(gradcams)):
    
    img = cv2.imread(img_paths[idx])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
    
    plt.subplot(3, 5, idx+1)
    plt.imshow(img)
    #plt.imshow(cams[idx], cmap="jet", alpha=.5)
    plt.axis('off')
    
    plt.subplot(3, 5, 5 + idx+1)
    plt.imshow(img)
    plt.imshow(gradcams[idx], cmap="jet", alpha=.5)
    plt.axis('off')
    
    #guided_gradcam = gradient[idx] * gradcams[idx][..., np.newaxis]
    guided_img = deprocess_image(guided_gradcams[idx])
    guided_img = cv2.addWeighted(cv2.cvtColor(img.astype('uint8'), cv2.COLOR_RGB2BGR), 0.5, guided_img, 1, 0) 
    
   
    plt.subplot(3, 5, 10 + idx+1)
    plt.imshow(guided_img)
    plt.axis('off')

# LIME

출처: https://github.com/marcotcr/lime/blob/master/doc/notebooks/Tutorial%20-%20Image%20Classification%20Keras.ipynb

In [0]:
!pip install lime

In [0]:
import os
import keras
from keras.applications import inception_v3 as inc_net
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
print('Notebook run using keras:', keras.__version__)

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd '/gdrive/My Drive/MVTEC_LEATHER/EXP/'

#### import model

In [0]:
model = keras.models.load_model('my_mvtec_model.h5')

In [0]:
def transform_img_fn(path_list):
    out = []
    for img_path in path_list:
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = x/255.0
        out.append(x)
    return np.vstack(out)

Let's see the top 5 prediction for some image

In [0]:
import glob, random
img_paths=[]
for i in range(1):
  img_path=random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/NG/*.jpg'))
  img_paths.append(img_path)
images = transform_img_fn(img_paths)
# I'm dividing by 2 and adding 0.5 because of how this Inception represents images
plt.imshow(images[0] / 2 + 0.5)
preds = model.predict(images)

#### Explanation
Now let's get an explanation

In [0]:
%load_ext autoreload
%autoreload 2
import os,sys
try:
    import lime
except:
    sys.path.append(os.path.join('..', '..')) # add the current directory
    import lime
from lime import lime_image

In [0]:
explainer = lime_image.LimeImageExplainer()

hide_color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels. Here, we set it to 0 (in the representation used by inception model, 0 means gray)

In [0]:
%%time
# Hide color is the color for a superpixel turned OFF. Alternatively, if it is NONE, the superpixel will be replaced by the average of its pixels
explanation = explainer.explain_instance(images[0], model.predict, top_labels=2, hide_color=0, num_samples=2000)

Image classifiers are a bit slow. Notice that an explanation on my Surface Book dGPU took 1min 29s

#### Now let's see the explanation for the top class
We can see the top 1 superpixels that are most positive towards the class with the rest of the image hidden

In [0]:
from skimage.segmentation import mark_boundaries

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Or with the rest of the image present:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))


We can also see the 'pros and cons' (pros in green, cons in red)

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Or the pros and cons that have weight at least 0.1

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=1000, hide_rest=False, min_weight=0.1)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))


Let's see the explanation for the second highest prediction
Most positive towards wombat:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

Pros and cons:

In [0]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[1], positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))

# SHAP

출처: https://slundberg.github.io/shap/notebooks/ImageNet%20VGG16%20Model%20with%20Keras.html

In [0]:
!pip install shap

In [0]:
import keras
from keras.preprocessing import image
import requests
from skimage.segmentation import slic
import matplotlib.pylab as plt
import numpy as np
import shap

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd '/gdrive/My Drive/MVTEC_LEATHER/EXP/'

#### import model

In [0]:
feature_names=['NG','OK']

In [0]:
model = keras.models.load_model('my_mvtec_model.h5')

#### NG

In [0]:
import glob, random

# load an image
file = random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/NG/*.jpg'))
img = image.load_img(file, target_size=(224, 224))
img_orig = image.img_to_array(img)/255.0

In [0]:
# segment the image so we don't have to explain every pixel
segments_slic = slic(img, n_segments=30, compactness=30, sigma=3)
#segments_slic = slic(img, n_segments=500, compactness=30, sigma=3)
plt.imshow(segments_slic);
plt.axis('off');

In [0]:
# define a function that depends on a binary mask representing if an image region is hidden
def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out
def f(z):
    return model.predict((mask_image(z, segments_slic, img_orig, 255))/255.0)
    #return model.predict(preprocess_input(mask_image(z, segments_slic, img_orig, 255)))

In [0]:
# use Kernel SHAP to explain the network's predictions
explainer = shap.KernelExplainer(f, np.zeros((1,50)))
shap_values = explainer.shap_values(np.ones((1,50)), nsamples=1000) # runs model 1000 times

In [0]:
# get the top predictions from the model
#preds = model.predict(preprocess_input(np.expand_dims(img_orig.copy(), axis=0)))
preds = model.predict(np.expand_dims(img_orig.copy(), axis=0))
top_preds = np.argsort(-preds)

In [0]:
def fill_segmentation(values, segmentation):
    out = np.zeros(segmentation.shape)
    for i in range(len(values)):
        out[segmentation == i] = values[i]
    return out

# plot our explanations
fig, axes = pl.subplots(nrows=1, ncols=3, figsize=(12,3))
inds = top_preds[0]
axes[0].imshow(img)
axes[0].axis('off')
max_val = np.max([np.max(np.abs(shap_values[i][:,:-1])) for i in range(len(shap_values))])
for i in range(2):
    m = fill_segmentation(shap_values[inds[i]][0], segments_slic)
    axes[i+1].set_title(feature_names[inds[i]])
    axes[i+1].imshow(img.convert('LA'), alpha=0.15)
    im = axes[i+1].imshow(m, cmap=cm, vmin=-max_val, vmax=max_val)
    axes[i+1].axis('off')
cb = fig.colorbar(im, ax=axes.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
cb.outline.set_visible(False)
pl.show()

#### OK

In [0]:
import glob, random

# load an image
file = random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/OK/*.jpg'))
img = image.load_img(file, target_size=(224, 224))
img_orig = image.img_to_array(img)/255.0



In [0]:
# segment the image so we don't have to explain every pixel
segments_slic = slic(img, n_segments=30, compactness=30, sigma=3)
#segments_slic = slic(img, n_segments=500, compactness=30, sigma=3)
plt.imshow(segments_slic);
plt.axis('off');

In [0]:
# define a function that depends on a binary mask representing if an image region is hidden
def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out
def f(z):
    return model.predict(preprocess_input(mask_image(z, segments_slic, img_orig, 255)))

In [0]:
# use Kernel SHAP to explain the network's predictions
explainer = shap.KernelExplainer(f, np.zeros((1,50)))
shap_values = explainer.shap_values(np.ones((1,50)), nsamples=2000) # runs model 1000 times

In [0]:
# get the top predictions from the model
preds = model.predict(preprocess_input(np.expand_dims(img_orig.copy(), axis=0)))
top_preds = np.argsort(-preds)

In [0]:
# make a color map
from matplotlib.colors import LinearSegmentedColormap
colors = []
for l in np.linspace(1,0,100):
    colors.append((245/255,39/255,87/255,l))
for l in np.linspace(0,1,100):
    colors.append((24/255,196/255,93/255,l))
cm = LinearSegmentedColormap.from_list("shap", colors)

In [0]:
def fill_segmentation(values, segmentation):
    out = np.zeros(segmentation.shape)
    for i in range(len(values)):
        out[segmentation == i] = values[i]
    return out

# plot our explanations
fig, axes = pl.subplots(nrows=1, ncols=3, figsize=(12,3))
inds = top_preds[0]
axes[0].imshow(img)
axes[0].axis('off')
max_val = np.max([np.max(np.abs(shap_values[i][:,:-1])) for i in range(len(shap_values))])
for i in range(2):
    m = fill_segmentation(shap_values[inds[i]][0], segments_slic)
    axes[i+1].set_title(feature_names[inds[i]])
    axes[i+1].imshow(img.convert('LA'), alpha=0.15)
    im = axes[i+1].imshow(m, cmap=cm, vmin=-max_val, vmax=max_val)
    axes[i+1].axis('off')
cb = fig.colorbar(im, ax=axes.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
cb.outline.set_visible(False)
pl.show()

### Original Source Code
출처: https://github.com/slundberg/shap

In [0]:
import glob, random

# load an image
file = random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/NG/*.jpg'))
X = image.load_img(file, target_size=(224, 224))
X = image.img_to_array(X)
X = np.expand_dims(X, axis=0)

In [0]:
import keras.backend as K
import numpy as np
import json
import shap

# load pre-trained model and choose two images to explain
#model = VGG16(weights='imagenet', include_top=True)
#X,y = shap.datasets.imagenet50()
to_explain = X
'''
# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)
'''
class_names=['NG','OK']

# explain how the input to the 344th layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)
e = shap.GradientExplainer(
    (model.layers[344].input, model.layers[-1].output),
    map2layer(X, 344),
    local_smoothing=0 # std dev of smoothing noise
)
shap_values,indexes = e.shap_values(map2layer(to_explain, 344), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[x])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

In [0]:
import keras.backend as K
import numpy as np
import json
import shap

# load pre-trained model and choose two images to explain
#model = VGG16(weights='imagenet', include_top=True)
#X,y = shap.datasets.imagenet50()
to_explain = X
'''
# load the ImageNet class names
url = "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
fname = shap.datasets.cache(url)
with open(fname) as f:
    class_names = json.load(f)
'''
class_names=['NG','OK']

# explain how the input to the 80h layer of the model explains the top two classes
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.layers[layer].input, feed_dict)
e = shap.GradientExplainer(
    (model.layers[80].input, model.layers[-1].output),
    map2layer(X, 80),
    local_smoothing=0 # std dev of smoothing noise
)
shap_values,indexes = e.shap_values(map2layer(to_explain, 80), ranked_outputs=2)

# get the names for the classes
index_names = np.vectorize(lambda x: class_names[x])(indexes)

# plot the explanations
shap.image_plot(shap_values, to_explain, index_names)

In [0]:
for i, l in enumerate(model.layers):
  print (i, l.name)

# IntegratedGradients

출처 ; https://github.com/hiranumn/IntegratedGradients/blob/master/examples/VGG%20example.ipynb

In [0]:
#@title IntegratedGradients Class import
################################################################
# Implemented by Naozumi Hiranuma (hiranumn@uw.edu)            #
#                                                              #
# Keras-compatible implmentation of Integrated Gradients       # 
# proposed in "Axiomatic attribution for deep neuron networks" #
# (https://arxiv.org/abs/1703.01365).                          #
#                                                              #
# Keywords: Shapley values, interpretable machine learning     #
################################################################

from __future__ import division, print_function
import numpy as np
from time import sleep
import sys
import keras.backend as K

from keras.models import Model, Sequential

'''
Integrated gradients approximates Shapley values by integrating partial
gradients with respect to input features from reference input to the
actual input. The following class implements the paper "Axiomatic attribution
for deep neuron networks".
'''
class integrated_gradients:
    # model: Keras model that you wish to explain.
    # outchannels: In case the model are multi tasking, you can specify which output you want explain .
    def __init__(self, model, outchannels=[], verbose=1):
    
        #get backend info (either tensorflow or theano)
        self.backend = K.backend()
        
        #load model supports keras.Model and keras.Sequential
        if isinstance(model, Sequential):
            self.model = model.model
        elif isinstance(model, Model):
            self.model = model
        else:
            print("Invalid input model")
            return -1
        
        #load input tensors
        self.input_tensors = []
        for i in self.model.inputs:
            self.input_tensors.append(i)
        # The learning phase flag is a bool tensor (0 = test, 1 = train)
        # to be passed as input to any Keras function that uses 
        # a different behavior at train time and test time.
        self.input_tensors.append(K.learning_phase())
        
        #If outputchanels are specified, use it.
        #Otherwise evalueate all outputs.
        self.outchannels = outchannels
        if len(self.outchannels) == 0: 
            if verbose: print("Evaluated output channel (0-based index): All")
            if K.backend() == "tensorflow":
                self.outchannels = range(self.model.output.shape[1]._value)
            elif K.backend() == "theano":
                self.outchannels = range(self.model.output._keras_shape[1])
        else:
            if verbose: 
                print("Evaluated output channels (0-based index):")
                print(','.join([str(i) for i in self.outchannels]))
                
        #Build gradient functions for desired output channels.
        self.get_gradients = {}
        if verbose: print("Building gradient functions")
        
        # Evaluate over all requested channels.
        for c in self.outchannels:
            # Get tensor that calculates gradient
            if K.backend() == "tensorflow":
                gradients = self.model.optimizer.get_gradients(self.model.output[:, c], self.model.input)
            if K.backend() == "theano":
                gradients = self.model.optimizer.get_gradients(self.model.output[:, c].sum(), self.model.input)
                
            # Build computational graph that computes the tensors given inputs
            self.get_gradients[c] = K.function(inputs=self.input_tensors, outputs=gradients)
            
            # This takes a lot of time for a big model with many tasks.
            # So lets print the progress.
            if verbose:
                sys.stdout.write('\r')
                sys.stdout.write("Progress: "+str(int((c+1)*1.0/len(self.outchannels)*1000)*1.0/10)+"%")
                sys.stdout.flush()
        # Done
        if verbose: print("\nDone.")
            
                
    '''
    Input: sample to explain, channel to explain
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample (defualted to 50).
    Output: list of numpy arrays to integrated over.
    '''
    def explain(self, sample, outc=0, reference=False, num_steps=50, verbose=0):
        
        # Each element for each input stream.
        samples = []
        numsteps = []
        step_sizes = []
        
        # If multiple inputs are present, feed them as list of np arrays. 
        if isinstance(sample, list):
            #If reference is present, reference and sample size need to be equal.
            if reference != False: 
                assert len(sample) == len(reference)
            for i in range(len(sample)):
                if reference == False:
                    _output = integrated_gradients.linearly_interpolate(sample[i], False, num_steps)
                else:
                    _output = integrated_gradients.linearly_interpolate(sample[i], reference[i], num_steps)
                samples.append(_output[0])
                numsteps.append(_output[1])
                step_sizes.append(_output[2])
        
        # Or you can feed just a single numpy arrray. 
        elif isinstance(sample, np.ndarray):
            _output = integrated_gradients.linearly_interpolate(sample, reference, num_steps)
            samples.append(_output[0])
            numsteps.append(_output[1])
            step_sizes.append(_output[2])
            
        # Desired channel must be in the list of outputchannels
        assert outc in self.outchannels
        if verbose: print("Explaning the "+str(self.outchannels[outc])+"th output.")
            
        # For tensorflow backend
        _input = []
        for s in samples:
            _input.append(s)
        _input.append(0)
        
        if K.backend() == "tensorflow": 
            gradients = self.get_gradients[outc](_input)
        elif K.backend() == "theano":
            gradients = self.get_gradients[outc](_input)
            if len(self.model.inputs) == 1:
                gradients = [gradients]
        
        explanation = []
        for i in range(len(gradients)):
            _temp = np.sum(gradients[i], axis=0)
            explanation.append(np.multiply(_temp, step_sizes[i]))
           
        # Format the return values according to the input sample.
        if isinstance(sample, list):
            return explanation
        elif isinstance(sample, np.ndarray):
            return explanation[0]
        return -1

    
    '''
    Input: numpy array of a sample
    Optional inputs:
        - reference: reference values (defaulted to 0s).
        - steps: # steps from reference values to the actual sample.
    Output: list of numpy arrays to integrate over.
    '''
    @staticmethod
    def linearly_interpolate(sample, reference=False, num_steps=50):
        # Use default reference values if reference is not specified
        if reference is False: reference = np.zeros(sample.shape);

        # Reference and sample shape needs to match exactly
        assert sample.shape == reference.shape

        # Calcuated stepwise difference from reference to the actual sample.
        ret = np.zeros(tuple([num_steps] +[i for i in sample.shape]))
        for s in range(num_steps):
            ret[s] = reference+(sample-reference)*(s*1.0/num_steps)

        return ret, num_steps, (sample-reference)*(1.0/num_steps)


In [0]:
from keras.applications.vgg16 import VGG16 as ModelTypeObj
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing import image


import matplotlib.pyplot as plt
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd '/gdrive/My Drive/MVTEC_LEATHER/EXP/'

### NG

#### Step 1. Load the model.

In [0]:
model = keras.models.load_model('my_mvtec_model.h5')

#### Step 2. Be sure to complie it and add an optimizer.

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

#### Step 3. Wrap it with integrated gradients.

In [0]:
ig = integrated_gradients(model)

#### Step 4. Obtain a sample to explain.

In [0]:
import glob, random

# load an image
file = random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/NG/*.jpg'))

img = image.load_img(file, target_size=(224, 224))
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.xticks([], [])
plt.yticks([], [])
plt.show()

In [0]:
# preprocess image
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0

# preprocess reference as well
ref = np.zeros((224, 224, 3))
ref = np.expand_dims(ref, axis=0)
#ref = preprocess_input(ref)
ref = ref/255.0

#### Step 5. Predict

In [0]:
labels = ['NG','OK']

In [0]:
pred = model.predict(x)

In [0]:
predicted = np.argmax(pred)
print ("Predicted label:", labels[predicted])

Step 6. Explain with respect to the true label.

In [0]:
exp = ig.explain(x[0], reference=ref[0], outc=predicted)

In [0]:
plt.figure(figsize=(8,4))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.xticks([], [])
plt.yticks([], [])
plt.title("Original image")

th = max(np.abs(np.min(exp)), np.abs(np.max(exp)))
plt.subplot(1, 2, 2)
plt.imshow(np.sum(exp, axis=2), cmap="seismic", vmin=-1*th, vmax=th)
plt.xticks([], [])
plt.yticks([], [])
plt.title("Explanation")
plt.show()

OK

#### Step 1. Load the model.

In [0]:
model = keras.models.load_model('my_mvtec_model.h5')

#### Step 2. Be sure to complie it and add an optimizer.

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy')

#### Step 3. Wrap it with integrated gradients.

In [0]:
ig = integrated_gradients(model)

#### Step 4. Obtain a sample to explain.

In [0]:
import glob, random

# load an image
file = random.choice(glob.glob('/gdrive/My Drive/MVTEC_LEATHER/EXP/Test/OK/*.jpg'))

img = image.load_img(file, target_size=(224, 224))
plt.figure(figsize=(2,2))
plt.imshow(img)
plt.xticks([], [])
plt.yticks([], [])
plt.show()

In [0]:
# preprocess image
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x/255.0

# preprocess reference as well
ref = np.zeros((224, 224, 3))
ref = np.expand_dims(ref, axis=0)
ref = ref/255.0

#### Step 5. Predict

In [0]:
labels = ['NG','OK']

In [0]:
pred = model.predict(x)

In [0]:
predicted = np.argmax(pred)
print ("Predicted label:", labels[predicted])

Step 6. Explain with respect to the true label.

In [0]:
exp = ig.explain(x[0], reference=ref[0], outc=predicted)

In [0]:
plt.figure(figsize=(4,2))
plt.subplot(1, 2, 1)
plt.imshow(img)
plt.xticks([], [])
plt.yticks([], [])
plt.title("Original image")

th = max(np.abs(np.min(exp)), np.abs(np.max(exp)))
plt.subplot(1, 2, 2)
plt.imshow(np.sum(exp, axis=2), cmap="seismic", vmin=-1*th, vmax=th)
plt.xticks([], [])
plt.yticks([], [])
plt.title("Explanation")
plt.show()

In [0]:
!nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
cd '/gdrive/My Drive/MVTEC_LEATHER/EXP/'